## Самостоятельная Работа 2

## Создание Таблицы

In [1]:
CREATE TABLE dbo.TestTable 
(
	id INT IDENTITY(1,1) NOT NULL, 
	manager NVARCHAR(50) NULL,
	department NVARCHAR(50) NULL,
	year INT NULL,
	sum MONEY NULL
)

Commands completed successfully.

Total execution time: 00:00:00.009

## Заполнение Данными

In [2]:
INSERT INTO dbo.TestTable (manager, department, year, sum)
VALUES
(N'Николаев_1', N'Администрация', 2014, 200),
(N'Николаев_2', N'Администрация', 2014, 300),
(N'Николаев_3', N'Отдел программирования', 2014, 150),
(N'Николаев_4', N'Отдел программирования', 2014, 200),
(N'Николаев_5', N'Отдел разработки', 2014, 250),
(N'Николаев_6', N'Отдел разработки', 2014, 300),
(N'Николаев_7', N'Отдел программирования', 2014, 300),
(N'Николаев_1', N'Администрация', 2015, 230),
(N'Николаев_2', N'Администрация', 2015, 200),
(N'Николаев_3', N'Отдел программирования', 2015, 200),
(N'Николаев_4', N'Отдел программирования', 2015, 300),
(N'Николаев_5', N'Отдел разработки', 2015, 200),
(N'Николаев_6', N'Отдел разработки', 2015, 250),
(N'Николаев_7', N'Отдел разработки', 2015, 350)

(14 rows affected)

Total execution time: 00:00:00.013

### Получить сумму расхода на оплату труда по отделам и по годам, и сначала давайте попробуем написать запрос с группировкой без использования оператора ROLLUP

In [3]:
SELECT department as otdel, year as god, SUM(sum) as result
FROM TestTable
GROUP BY department, year
ORDER BY department, year

(6 rows affected)

Total execution time: 00:00:00.025

otdel,god,result
Администрация,2014,500.00
Администрация,2015,430.00
Отдел программирования,2014,650.00
Отдел программирования,2015,500.00
Отдел разработки,2014,550.00
Отдел разработки,2015,800.00


### Получим общий расход по каждому отделу. Для этих целей мы можем использовать оператор ROLLUP

In [4]:
SELECT department, year, SUM(sum) AS result
FROM dbo.TestTable
GROUP BY ROLLUP(department, year)
ORDER BY department, year

(10 rows affected)

Total execution time: 00:00:00.008

department,year,result
NULL,NULL,3430.00
Администрация,NULL,930.00
Администрация,2014,500.00
Администрация,2015,430.00
Отдел программирования,NULL,1150.00
Отдел программирования,2014,650.00
Отдел программирования,2015,500.00
Отдел разработки,NULL,1350.00
Отдел разработки,2014,550.00
Отдел разработки,2015,800.00


### Группировка по отделам с общим итогом

In [5]:
Select department, year, SUM(sum) as result
	from dbo.TestTable
group by 
rollup (department, year)

(10 rows affected)

Total execution time: 00:00:00.008

department,year,result
Администрация,2014,500.00
Администрация,2015,430.00
Администрация,NULL,930.00
Отдел программирования,2014,650.00
Отдел программирования,2015,500.00
Отдел программирования,NULL,1150.00
Отдел разработки,2014,550.00
Отдел разработки,2015,800.00
Отдел разработки,NULL,1350.00
NULL,NULL,3430.00


### Сгруппируем и промежуточные итоги для года и отдела

In [6]:
Select department, year, SUM(sum) as result
	from dbo.TestTable
group by 
cube (department, year)

(12 rows affected)

Total execution time: 00:00:00.014

department,year,result
Администрация,2014,500.00
Отдел программирования,2014,650.00
Отдел разработки,2014,550.00
NULL,2014,1700.00
Администрация,2015,430.00
Отдел программирования,2015,500.00
Отдел разработки,2015,800.00
NULL,2015,1730.00
NULL,NULL,3430.00
Администрация,NULL,930.00


### Группируем множества

In [7]:
Select department, year, SUM(sum) as result
	from dbo.TestTable
group by 
grouping sets (department, year)

(5 rows affected)

Total execution time: 00:00:00.010

department,year,result
NULL,2014,1700.00
NULL,2015,1730.00
Администрация,NULL,930.00
Отдел программирования,NULL,1150.00
Отдел разработки,NULL,1350.00


### Условное объединение

In [8]:
Select department,
    ISNULL(cast(year as varchar(30)),
			case when GROUPING(year)=1 and GROUPING(department)=0
			then '' else '' end) as year,
    SUM(sum) as result,
    grouping (department) as groupingDepartment,
    grouping(year) as groupingYear
from dbo.TestTable
group by 
rollup (department, year)

(10 rows affected)

Total execution time: 00:00:00.011

department,year,result,groupingDepartment,groupingYear
Администрация,2014,500.00,0,0
Администрация,2015,430.00,0,0
Администрация,,930.00,0,1
Отдел программирования,2014,650.00,0,0
Отдел программирования,2015,500.00,0,0
Отдел программирования,,1150.00,0,1
Отдел разработки,2014,550.00,0,0
Отдел разработки,2015,800.00,0,0
Отдел разработки,,1350.00,0,1
NULL,,3430.00,1,1
